In [ ]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/colab_setup.sh -O - | bash

--2021-10-02 15:12:58--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-10-02 15:12:58 (25.7 MB/s) - written to stdout [1608/1608]

setup Colab for PySpark 3.0.2 and Spark NLP 3.1.0
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  R

In [ ]:
import sparknlp
spark= sparknlp.start()

In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.sql import functions as F
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType

In [ ]:
!wget -q -O news_category_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test.csv

In [ ]:
news_df= spark.read\
    .option("header", True)\
    .csv("/content/news_category_test.csv")\
    .withColumnRenamed("description", "text")

news_df.show(truncate=40)

+--------+----------------------------------------+
|category|                                    text|
+--------+----------------------------------------+
|Business|Unions representing workers at Turner...|
|Sci/Tech| TORONTO, Canada    A second team of ...|
|Sci/Tech| A company founded by a chemistry res...|
|Sci/Tech| It's barely dawn when Mike Fitzpatri...|
|Sci/Tech| Southern California's smog fighting ...|
|Sci/Tech|"The British Department for Education...|
|Sci/Tech|"confessed author of the Netsky and S...|
|Sci/Tech|\\FOAF/LOAF  and bloom filters have a...|
|Sci/Tech|"Wiltshire Police warns about ""phish...|
|Sci/Tech|In its first two years, the UK's dedi...|
|Sci/Tech| A group of technology companies  inc...|
|Sci/Tech| Apple Computer Inc.&lt;AAPL.O&gt; on...|
|Sci/Tech| Free Record Shop, a Dutch music  ret...|
|Sci/Tech|A giant 100km colony of ants  which h...|
|Sci/Tech|            "Dolphin groups, or ""pods""|
|Sci/Tech|Tyrannosaurus rex achieved its massiv...|
|Sci/Tech|  

In [ ]:
lemmatizer= LemmatizerModel.pretrained("lemma_antbnc","en") \
      .setInputCols(["token"])\
      .setOutputCol("lemma")

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [ ]:
!cd ~/cache_pretrained && ls -1

lemma_antbnc_en_2.0.2_2.4_1556480454569


In [ ]:
documentAssembler= DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer= Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

stemmer= Stemmer()\
    .setInputCols(["token"])\
    .setOutputCol("stem")

nlpPipeline= Pipeline(stages=[ 
                              documentAssembler,
                              tokenizer,
                              stemmer,
                              lemmatizer
])

empty_df= spark.createDataFrame([[" "]]).toDF("text")
pipeline_model= nlpPipeline.fit(empty_df)

result= pipeline_model.transform(news_df)

In [ ]:
result.show(5, truncate=40)

+--------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|category|                                    text|                                document|                                   token|                                    stem|                                   lemma|
+--------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|Business|Unions representing workers at Turner...|[[document, 0, 126, Unions representi...|[[token, 0, 5, Unions, [sentence -> 0...|[[token, 0, 5, union, [sentence -> 0]...|[[token, 0, 5, Unions, [sentence -> 0...|
|Sci/Tech| TORONTO, Canada    A second team of ...|[[document, 0, 222,  TORONTO, Canada ...|[[token, 1, 7, TORONTO, [sentence -> ...|[[t

In [ ]:
result.select("token.result", "stem.result", "lemma.result")\
    .toDF("token", "stem", "lemma").show(10, truncate=70)

+----------------------------------------------------------------------+----------------------------------------------------------------------+----------------------------------------------------------------------+
|                                                                 token|                                                                  stem|                                                                 lemma|
+----------------------------------------------------------------------+----------------------------------------------------------------------+----------------------------------------------------------------------+
|[Unions, representing, workers, at, Turner, Newall, say, they, are,...|[union, repres, worker, at, turner, newal, sai, thei, ar, ', disapp...|[Unions, represent, worker, at, Turner, Newall, say, they, be, ', d...|
|[TORONTO, ,, Canada, A, second, team, of, rocketeers, competing, fo...|[toronto, ,, canada, a, second, team, of, rocket, compet, for, the,.

In [ ]:
result.select(F.explode(F.arrays_zip("token.result", "stem.result", "lemma.result")).alias("col"))\
    .select(F.expr("col['0']").alias("token"),
            F.expr("col['1']").alias("stem"),
            F.expr("col['2']").alias("lemma")).show(10, truncate=False)

+------------+------+---------+
|token       |stem  |lemma    |
+------------+------+---------+
|Unions      |union |Unions   |
|representing|repres|represent|
|workers     |worker|worker   |
|at          |at    |at       |
|Turner      |turner|Turner   |
|Newall      |newal |Newall   |
|say         |sai   |say      |
|they        |thei  |they     |
|are         |ar    |be       |
|'           |'     |'        |
+------------+------+---------+
only showing top 10 rows



In [ ]:
pos= PerceptronModel.pretrained("pos_anc", "en")\
    .setInputCols(["document", "token"])\
    .setOutputCol("pos")

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]


In [ ]:
nlpPipeline= Pipeline(stages=[ 
                              documentAssembler, 
                              tokenizer,
                              stemmer,
                              lemmatizer,
                              pos
])

empty_df= spark.createDataFrame([[" "]]).toDF("text")
pipeline_model= nlpPipeline.fit(empty_df)

result= pipeline_model.transform(news_df)

In [ ]:
!cd ~/cache_pretrained && ls -1

lemma_antbnc_en_2.0.2_2.4_1556480454569
pos_anc_en_3.0.0_3.0_1614962126490


In [ ]:
result.show(5, truncate=50)

+--------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|category|                                              text|                                          document|                                             token|                                              stem|                                             lemma|                                               pos|
+--------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|Business|Unions representing workers at Turner  

In [ ]:
df= result.select(F.explode(F.arrays_zip("token.result", "token.begin", "token.end", "stem.result", "lemma.result", "pos.result")).alias("col"))\
    .select(F.expr("col['0']").alias("token"),
            F.expr("col['1']").alias("begin"),
            F.expr("col['2']").alias("end"),
            F.expr("col['3']").alias("stem"),
            F.expr("col['4']").alias("lemma"),
            F.expr("col['5']").alias("pos")).toPandas()
df.head()

,token,begin,end,stem,lemma,pos
0,Unions,0,5,union,Unions,NNP
1,representing,7,18,repres,represent,VBG
2,workers,20,26,worker,worker,NNS
3,at,28,29,at,at,IN
4,Turner,31,36,turner,Turner,NNP


In [ ]:
chunker= Chunker()\
    .setInputCols(["document", "pos"])\
    .setOutputCol("chunk")\
    .setRegexParsers(["<NNP>+", "<DT>?<JJ>*<NN>"])

In [ ]:
chunker.extractParamMap()

{Param(parent='Chunker_8e246c808a05', name='inputCols', doc='previous annotations columns, if renamed'): ['document'],
 Param(parent='Chunker_8e246c808a05', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='Chunker_8e246c808a05', name='outputCol', doc='output annotation column. can be left default.'): 'chunk',
 Param(parent='Chunker_8e246c808a05', name='regexParsers', doc='an array of grammar based chunk parsers'): ['<NNP>+',
  '<DT>?<JJ>*<NN>']}

In [ ]:
nlpPipeline= Pipeline(stages=[ 
                              documentAssembler, 
                              tokenizer,
                              stemmer,
                              lemmatizer,
                              pos,
                              chunker
])
empty_df= spark.createDataFrame([[" "]]).toDF("text")
pipeline_model= nlpPipeline.fit(empty_df)

result= pipeline_model.transform(news_df.limit(100))

In [ ]:
result.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|category|                text|            document|               token|                stem|               lemma|                 pos|               chunk|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Business|Unions representi...|[[document, 0, 12...|[[token, 0, 5, Un...|[[token, 0, 5, un...|[[token, 0, 5, Un...|[[pos, 0, 5, NNP,...|[[chunk, 0, 5, Un...|
|Sci/Tech| TORONTO, Canada ...|[[document, 0, 22...|[[token, 1, 7, TO...|[[token, 1, 7, to...|[[token, 1, 7, TO...|[[pos, 1, 7, NNP,...|[[chunk, 1, 7, TO...|
|Sci/Tech| A company founde...|[[document, 0, 20...|[[token, 1, 1, A,...|[[token, 1, 1, a,...|[[token, 1, 1, A,...|[[pos, 1, 1, DT, ...|[[chunk, 52, 61, ...|
|Sci/Tech| It's barely dawn...|[[document, 0, 26...|

In [ ]:
df= result.select(F.explode(F.arrays_zip("chunk.result", "chunk.begin", "chunk.end")).alias("col"))\
    .select(F.expr("col['0']").alias("chunk"),
            F.expr("col['1']").alias("begin"),
            F.expr("col['2']").alias("end")).toPandas()

df.head()

,chunk,begin,end
0,Unions,0,5
1,Turner Newall,31,45
2,Federal Mogul,113,125
3,stricken,92,99
4,parent,101,106


In [ ]:
stopwords_cleaner= StopWordsCleaner.pretrained("stopwords_en", "en")\
    .setInputCols(["token"])\
    .setOutputCol("cleaned")\
    .setCaseSensitive(False)


stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[OK!]


In [ ]:
!cd ~/cache_pretrained && ls -1

lemma_antbnc_en_2.0.2_2.4_1556480454569
pos_anc_en_3.0.0_3.0_1614962126490
stopwords_en_en_2.5.4_2.4_1594742439135


In [ ]:
stopwords_cleaner.getStopWords()[:10]

['a',
 "a's",
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after']

In [ ]:
token_assembler= TokenAssembler()\
    .setInputCols(["document", "cleaned"])\
    .setOutputCol("cleaned_tokens")

nlpPipeline= Pipeline(stages=[ 
                              documentAssembler,
                              tokenizer,
                              stopwords_cleaner,
                              token_assembler
])

nlp_model= nlpPipeline.fit(empty_df)

In [ ]:
#lightpipeline version:
light_model= LightPipeline(nlp_model)
light_result= light_model.annotate("Peter Parker is a nice person and a friend of mine.")
light_result["cleaned_tokens"]

['Peter Parker nice person friend mine.']

In [ ]:
result= nlp_model.transform(news_df)
result.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|category|                text|            document|               token|             cleaned|      cleaned_tokens|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Business|Unions representi...|[[document, 0, 12...|[[token, 0, 5, Un...|[[token, 0, 5, Un...|[[document, 0, 97...|
|Sci/Tech| TORONTO, Canada ...|[[document, 0, 22...|[[token, 1, 7, TO...|[[token, 1, 7, TO...|[[document, 0, 16...|
|Sci/Tech| A company founde...|[[document, 0, 20...|[[token, 1, 1, A,...|[[token, 3, 9, co...|[[document, 0, 15...|
|Sci/Tech| It's barely dawn...|[[document, 0, 26...|[[token, 1, 4, It...|[[token, 6, 11, b...|[[document, 0, 17...|
|Sci/Tech| Southern Califor...|[[document, 0, 17...|[[token, 1, 8, So...|[[token, 1, 8, So...|[[document, 0, 13...|
|Sci/Tech|"The British Depa...|[[document, 0, 10...|[[token, 0, 0, ",...

In [ ]:
spellModel= ContextSpellCheckerModel.pretrained("spellcheck_dl")\
    .setInputCols(["token"])\
    .setOutputCol("spell_checked")

finisher= Finisher()\
    .setInputCols(["spell_checked"])\
  
pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           tokenizer,
                           spellModel,
                           finisher
])

empty_df= spark.createDataFrame([[" "]]).toDF("text")
pipeline_model= pipeline.fit(empty_df)



spellcheck_dl download started this may take some time.
Approximate size to download 111.4 MB
[OK!]


In [ ]:
lp= LightPipeline(pipeline_model)

In [ ]:
lp.annotate("Plaese alliow me tao introdduce myhelf, I am a man of waelth und tiaste")

{'spell_checked': ['Please',
  'allow',
  'me',
  'to',
  'introduce',
  'myself',
  ',',
  'I',
  'am',
  'a',
  'man',
  'of',
  'wealth',
  'und',
  'taste']}

In [ ]:
result= pipeline_model.transform(news_df.limit(100))

In [ ]:
result.show()

+--------+--------------------+----------------------+
|category|                text|finished_spell_checked|
+--------+--------------------+----------------------+
|Business|Unions representi...|  [Unions, represen...|
|Sci/Tech| TORONTO, Canada ...|  [TORONTO, ,, Cana...|
|Sci/Tech| A company founde...|  [A, company, foun...|
|Sci/Tech| It's barely dawn...|  [It's, barely, da...|
|Sci/Tech| Southern Califor...|  [Southern, Califo...|
|Sci/Tech|"The British Depa...|  [", The, British,...|
|Sci/Tech|"confessed author...|  [", confused, aut...|
|Sci/Tech|\\FOAF/LOAF  and ...|  [\\FOAF/LOAF, and...|
|Sci/Tech|"Wiltshire Police...|  [", Wilshire, Pol...|
|Sci/Tech|In its first two ...|  [In, its, first, ...|
|Sci/Tech| A group of techn...|  [A, group, of, te...|
|Sci/Tech| Apple Computer I...|  [Apple, Computer,...|
|Sci/Tech| Free Record Shop...|  [Free, Record, Sh...|
|Sci/Tech|A giant 100km col...|  [A, giant, 100mm,...|
|Sci/Tech|"Dolphin groups, ...|  [", Dolphin, grou...|
|Sci/Tech|

In [ ]:
examples = ['We will go to swimming if the ueather is sunny.',\
    "I have a black ueather jacket, so nice.",\
    "I introduce you to my sister, she is called ueather."]

spark_df= spark.createDataFrame(examples, StringType()).toDF("text")
result= pipeline_model.transform(spark_df)

In [ ]:
result.show(truncate=False)

+----------------------------------------------------+-------------------------------------------------------------------+
|text                                                |finished_spell_checked                                             |
+----------------------------------------------------+-------------------------------------------------------------------+
|We will go to swimming if the ueather is sunny.     |[We, will, go, to, swimming, if, the, weather, is, sunny, .]       |
|I have a black ueather jacket, so nice.             |[I, have, a, black, leather, jacket, ,, so, nice, .]               |
|I introduce you to my sister, she is called ueather.|[I, introduce, you, to, my, sister, ,, she, is, called, Heather, .]|
+----------------------------------------------------+-------------------------------------------------------------------+



In [ ]:
pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           tokenizer,
                           spellModel,
                      
])

empty_df= spark.createDataFrame([[" "]]).toDF("text")
pipeline_model= pipeline.fit(empty_df)

In [ ]:
spark_df= spark.createDataFrame(examples, StringType()).toDF("text")
pipeline_model= pipeline.fit(empty_df)
result= pipeline_model.transform(spark_df)

In [ ]:
result.show()

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|       spell_checked|
+--------------------+--------------------+--------------------+--------------------+
|We will go to swi...|[[document, 0, 46...|[[token, 0, 1, We...|[[token, 0, 1, We...|
|I have a black ue...|[[document, 0, 38...|[[token, 0, 0, I,...|[[token, 0, 0, I,...|
|I introduce you t...|[[document, 0, 51...|[[token, 0, 0, I,...|[[token, 0, 0, I,...|
+--------------------+--------------------+--------------------+--------------------+



In [ ]:
result.select("spell_checked.result").show(truncate=False)

+-------------------------------------------------------------------+
|result                                                             |
+-------------------------------------------------------------------+
|[We, will, go, to, swimming, if, the, weather, is, sunny, .]       |
|[I, have, a, black, leather, jacket, ,, so, nice, .]               |
|[I, introduce, you, to, my, sister, ,, she, is, called, Heather, .]|
+-------------------------------------------------------------------+



## Word Embedding

In [ ]:
glove_embedding= WordEmbeddingsModel.pretrained("glove_100d")\
    .setInputCols(["document", "token"])\
    .setOutputCol("glove")

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
!cd ~/cache_pretrained && ls -1

glove_100d_en_2.4.0_2.4_1579690104032


In [ ]:
news_df= spark.read\
    .option("header", True)\
    .csv("/content/news_category_test.csv")\
    .withColumnRenamed("description", "text")

In [ ]:
documentAssembler= DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer= Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           tokenizer,
                           glove_embedding
])

empty_df= spark.createDataFrame([[" "]]).toDF("text")
pipeline_model= pipeline.fit(empty_df)
result= pipeline_model.transform(news_df.limit(50))

In [ ]:
result.show(5, truncate=60)

+--------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+
|category|                                                        text|                                                    document|                                                       token|                                                       glove|
+--------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+------------------------------------------------------------+
|Business|Unions representing workers at Turner   Newall say they a...|[[document, 0, 126, Unions representing workers at Turner...|[[token, 0, 5, Unions, [sentence -> 0], []], [token, 7, 1...|[[word_embeddings, 0, 5, Unions, [isOOV ->

In [ ]:
result.select(F.explode(F.arrays_zip("token.result", "glove.embeddings")).alias("col"))\
    .select(F.expr("col['0']").alias("token"),
            F.expr("col['1']").alias("glove_embedded")).show(truncate=False)

+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|token       |glove_embed

In [ ]:
elmo_embedding= ElmoEmbeddings.pretrained("elmo")\
    .setInputCols(["document", "token"])\
    .setOutputCol("elmo")\
    .setPoolingLayer("elmo") #default -> elmo; other options-> word_emb, lstm_outputs1, lstm_outputs2

elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]


In [ ]:
pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           tokenizer,
                           elmo_embedding
])

empty_df= spark.createDataFrame([[" "]]).toDF("text")
pipeline_model= pipeline.fit(empty_df)
result= pipeline_model.transform(news_df.limit(50))

In [ ]:
result.select(F.explode(F.arrays_zip("token.result", "elmo.embeddings")).alias("col"))\
    .select(F.expr("col['0']").alias("token"),
            F.expr("col['1']").alias("elmo_embedding")).show(7, truncate=False)

+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
bert_embedding= BertEmbeddings.pretrained("bert_base_uncased")\
    .setInputCols(["document","token"])\
    .setOutputCol("bert_embedding")\

bert_base_uncased download started this may take some time.
Approximate size to download 392.5 MB
[OK!]


In [ ]:
bert_embedding.extractParamMap()

{Param(parent='BERT_EMBEDDINGS_4fbd72cbda5a', name='batchSize', doc='Size of every batch'): 8,
 Param(parent='BERT_EMBEDDINGS_4fbd72cbda5a', name='caseSensitive', doc='whether to ignore case in tokens for embeddings matching'): False,
 Param(parent='BERT_EMBEDDINGS_4fbd72cbda5a', name='dimension', doc='Number of embedding dimensions'): 768,
 Param(parent='BERT_EMBEDDINGS_4fbd72cbda5a', name='inputCols', doc='previous annotations columns, if renamed'): ['document',
  'token'],
 Param(parent='BERT_EMBEDDINGS_4fbd72cbda5a', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='BERT_EMBEDDINGS_4fbd72cbda5a', name='maxSentenceLength', doc='Max sentence length to process'): 128,
 Param(parent='BERT_EMBEDDINGS_4fbd72cbda5a', name='outputCol', doc='output annotation column. can be left default.'): 'bert_embedding',
 Param(parent='BERT_EMBEDDINGS_4fbd72cbda5a', name='storageRef', doc='unique reference name for identification'): 'bert_

In [ ]:
pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           tokenizer,
                           bert_embedding
])

empty_df= spark.createDataFrame([[" "]]).toDF("text")
pipeline_model= pipeline.fit(empty_df)
result= pipeline_model.transform(news_df.limit(50))

In [ ]:
result.select(F.explode(F.arrays_zip("token.result", "bert_embedding.embeddings")).alias("col"))\
    .select(F.expr("col['0']").alias("token"),
            F.expr("col['1']").alias("bert_embedding")).show(7, truncate=False)

+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
entities = ['parent firm', 'economy', 'amino acids']
with open("entities.txt" , "w") as f:
  for i in entities:
    f.write(i+ "\n")


In [ ]:
documentAssembler= DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer= Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

entity_extractor= TextMatcher()\
    .setInputCols(["document", "token"])\
    .setOutputCol("entities")\
    .setCaseSensitive(False)\
    .setEntities("entities.txt")\
    .setEntityValue("entities")

In [ ]:
nlpPipeline= Pipeline(stages=[ 
                              documentAssembler,
                              tokenizer,
                              entity_extractor
])

empty_df= spark.createDataFrame([[" "]]).toDF("text")
model= nlpPipeline.fit(empty_df)
result= model.transform(news_df.limit(10))

In [ ]:
result.show(5, truncate=20)

+--------+--------------------+--------------------+--------------------+--------------------+
|category|                text|            document|               token|            entities|
+--------+--------------------+--------------------+--------------------+--------------------+
|Business|Unions representi...|[[document, 0, 12...|[[token, 0, 5, Un...|[[chunk, 101, 111...|
|Sci/Tech| TORONTO, Canada ...|[[document, 0, 22...|[[token, 1, 7, TO...|                  []|
|Sci/Tech| A company founde...|[[document, 0, 20...|[[token, 1, 1, A,...|[[chunk, 165, 175...|
|Sci/Tech| It's barely dawn...|[[document, 0, 26...|[[token, 1, 4, It...|                  []|
|Sci/Tech| Southern Califor...|[[document, 0, 17...|[[token, 1, 8, So...|                  []|
+--------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
result.select("text","entities.result").show(3, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|text                                                                                                                                                                                                                           |result       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.                                                                                                |[parent firm]|
| TORONTO, Canada    A second team of ro

### Chunk Embedding

In [ ]:
word_embedding= WordEmbeddingsModel.pretrained("glove_100d")\
    .setInputCols(["document", "token"])\
    .setOutputCol("word_embedding")

chunk_embedding= ChunkEmbeddings()\
    .setInputCols(["entities","word_embedding"])\
    .setOutputCol("chunk_embedding")\
    .setPoolingStrategy("Average") #other option is: sum

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
pipeline= Pipeline(stages=[ 
                           documentAssembler, 
                           tokenizer,
                           entity_extractor,
                           word_embedding,
                           chunk_embedding
])

pipelineModel= pipeline.fit(empty_df)
result= pipelineModel.transform(news_df.limit(10))

In [ ]:
result.show(5, truncate=20)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|category|                text|            document|               token|            entities|      word_embedding|     chunk_embedding|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Business|Unions representi...|[[document, 0, 12...|[[token, 0, 5, Un...|[[chunk, 101, 111...|[[word_embeddings...|[[word_embeddings...|
|Sci/Tech| TORONTO, Canada ...|[[document, 0, 22...|[[token, 1, 7, TO...|                  []|[[word_embeddings...|                  []|
|Sci/Tech| A company founde...|[[document, 0, 20...|[[token, 1, 1, A,...|[[chunk, 165, 175...|[[word_embeddings...|[[word_embeddings...|
|Sci/Tech| It's barely dawn...|[[document, 0, 26...|[[token, 1, 4, It...|                  []|[[word_embeddings...|                  []|
|Sci/Tech| Southern Califor...|[[document

In [ ]:
result.select(F.explode(F.arrays_zip("entities.result" ,"chunk_embedding.embeddings")).alias("col"))\
    .select(F.expr("col['0']").alias("entities"),
            F.expr("col['1']").alias("chunk_embeddings")).show()

+-----------+--------------------+
|   entities|    chunk_embeddings|
+-----------+--------------------+
|parent firm|[0.45683652, -0.1...|
|amino acids|[-0.3861, 0.05440...|
+-----------+--------------------+



In [ ]:
!cd ~/cache_pretrained && pwd

/root/cache_pretrained


In [ ]:
!cd ~/cache_pretrained && ls -1

glove_100d_en_2.4.0_2.4_1579690104032
lemma_antbnc_en_2.0.2_2.4_1556480454569


## Loading Models From Local

In [ ]:
word_embedding= WordEmbeddingsModel.load("/root/cache_pretrained/glove_100d_en_2.4.0_2.4_1579690104032")\
    .setInputCols(["document", "token"])\
    .setOutputCol("glove_embedding")

In [ ]:
use= UniversalSentenceEncoder.pretrained("tfhub_use", "en")\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [ ]:
pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           use
])
model= pipeline.fit(empty_df)
result= model.transform(news_df.limit(10))

In [ ]:
result.show(truncate=50)

+--------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|category|                                              text|                                          document|                               sentence_embeddings|
+--------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|Business|Unions representing workers at Turner   Newall ...|[[document, 0, 126, Unions representing workers...|[[sentence_embeddings, 0, 126, Unions represent...|
|Sci/Tech| TORONTO, Canada    A second team of rocketeers...|[[document, 0, 222,  TORONTO, Canada    A secon...|[[sentence_embeddings, 0, 222,  TORONTO, Canada...|
|Sci/Tech| A company founded by a chemistry researcher at...|[[document, 0, 209,  A company founded by a che...|[[sentence_embeddings, 0, 209,  A company found...|
|Sci/Tech| It's 

In [ ]:
result.select(F.explode(F.arrays_zip("document.result","sentence_embeddings.embeddings")).alias("col"))\
      .select(F.expr("col['0']").alias("document"),
              F.expr("col['1']").alias("use_embedding")).show(2,truncate=180)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                            document|                                                                                                                                                                       use_embedding|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------

In [ ]:
spark_df= spark.read.text("/content/example_text.txt").toDF("text")
spark_df.show(truncate=False)

+-------------------------------------------------------+
|text                                                   |
+-------------------------------------------------------+
|YASAMAK SAKAYA GELMEZ,                                 |
|BÜYÜK BİR CİDDİYETLE YASAYACAKSIN.                     |
|BİR SİNCAP GİBİ MESELA,                                |
|YANI, YASAMIN DIŞINDA VE ÖTESİNDE HİÇBİR ŞEY BEKLEMEDEN|
|YANI, BÜTÜN İŞİN GÜCÜN YASAMAK OLACAK.                 |
+-------------------------------------------------------+



In [ ]:
result_1= model.transform(spark_df)
result_1.show()

+--------------------+--------------------+--------------------+
|                text|            document| sentence_embeddings|
+--------------------+--------------------+--------------------+
|YASAMAK SAKAYA GE...|[[document, 0, 21...|[[sentence_embedd...|
|BÜYÜK BİR CİDDİYE...|[[document, 0, 33...|[[sentence_embedd...|
|BİR SİNCAP GİBİ M...|[[document, 0, 22...|[[sentence_embedd...|
|YANI, YASAMIN DIŞ...|[[document, 0, 54...|[[sentence_embedd...|
|YANI, BÜTÜN İŞİN ...|[[document, 0, 37...|[[sentence_embedd...|
+--------------------+--------------------+--------------------+



In [ ]:
result_1.select(F.explode(F.arrays_zip("document.result" ,"sentence_embeddings.embeddings")).alias())\
    .select(F.expr("col['0']").alias("document"),
            F.expr("col['1']").alias("sentence_embedding")).show(truncate=False)

+-------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Getting Sentence Embeddings From Word Embeddings

In [ ]:
!cd ~/cache_pretrained && pwd

/root/cache_pretrained


In [ ]:
!cd ~/cache_pretrained && ls -1

glove_100d_en_2.4.0_2.4_1579690104032
lemma_antbnc_en_2.0.2_2.4_1556480454569
tfhub_use_en_2.4.0_2.4_1587136330099


In [ ]:
word_embedding= WordEmbeddingsModel.load("/root/cache_pretrained/glove_100d_en_2.4.0_2.4_1579690104032")\
    .setInputCols(["document", "token"])\
    .setOutputCol("word_embedding")

sentence_embedding= SentenceEmbeddings()\
    .setInputCols(["document", "word_embedding"])\
    .setOutputCol("sentence_embedding")\
    .setPoolingStrategy("AVERAGE") # or SUM

pipeline= Pipeline(stages=[
                           documentAssembler,
                           tokenizer,
                           word_embedding,
                           sentence_embedding
 ])

model= pipeline.fit(empty_df)
result= model.transform(spark_df)

In [ ]:
result.show(truncate=20)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|      word_embedding|  sentence_embedding|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|YASAMAK SAKAYA GE...|[[document, 0, 21...|[[token, 0, 6, YA...|[[word_embeddings...|[[sentence_embedd...|
|BÜYÜK BİR CİDDİYE...|[[document, 0, 33...|[[token, 0, 4, BÜ...|[[word_embeddings...|[[sentence_embedd...|
|BİR SİNCAP GİBİ M...|[[document, 0, 22...|[[token, 0, 2, Bİ...|[[word_embeddings...|[[sentence_embedd...|
|YANI, YASAMIN DIŞ...|[[document, 0, 54...|[[token, 0, 3, YA...|[[word_embeddings...|[[sentence_embedd...|
|YANI, BÜTÜN İŞİN ...|[[document, 0, 37...|[[token, 0, 3, YA...|[[word_embeddings...|[[sentence_embedd...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [ ]:
result.select(F.explode(F.arrays_zip("document.result", "sentence_embedding.embeddings")).alias("col"))\
    .select(F.expr("col['0']").alias("document"),
            F.expr("col['1']").alias("sentence_embedding")).show(truncate=40)

+----------------------------------------+----------------------------------------+
|                                document|                      sentence_embedding|
+----------------------------------------+----------------------------------------+
|                  YASAMAK SAKAYA GELMEZ,|[0.043702498, 0.11233, 0.012634993, -...|
|      BÜYÜK BİR CİDDİYETLE YASAYACAKSIN.|[-0.033145998, -0.124207996, 0.04482,...|
|                 BİR SİNCAP GİBİ MESELA,|[-0.021534, 0.022106, 0.119624, -0.10...|
|YANI, YASAMIN DIŞINDA VE ÖTESİNDE HİÇ...|[-0.23753555, -0.018644445, -0.055696...|
|  YANI, BÜTÜN İŞİN GÜCÜN YASAMAK OLACAK.|[-0.25997, -0.018052502, 0.07244125, ...|
+----------------------------------------+----------------------------------------+



## NERDL Model

In [ ]:
documentAssembler= DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer= Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

word_embedding= WordEmbeddingsModel.pretrained("glove_100d")\
    .setInputCols(["document", "token"])\
    .setOutputCol("embedding")

public_ner= NerDLModel.pretrained("ner_dl", "en")\
    .setInputCols(["document", "token", "embedding"])\
    .setOutputCol("ner")

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]


In [ ]:
public_ner.extractParamMap()

{Param(parent='NerDLModel_d4424c9af5f4', name='batchSize', doc='Size of every batch'): 32,
 Param(parent='NerDLModel_d4424c9af5f4', name='classes', doc='get the tags used to trained this NerDLModel'): ['O',
  'B-ORG',
  'B-LOC',
  'B-PER',
  'I-PER',
  'I-ORG',
  'B-MISC',
  'I-LOC',
  'I-MISC'],
 Param(parent='NerDLModel_d4424c9af5f4', name='includeConfidence', doc='whether to include confidence scores in annotation metadata'): False,
 Param(parent='NerDLModel_d4424c9af5f4', name='inputCols', doc='previous annotations columns, if renamed'): ['document',
  'token',
  'embedding'],
 Param(parent='NerDLModel_d4424c9af5f4', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='NerDLModel_d4424c9af5f4', name='outputCol', doc='output annotation column. can be left default.'): 'ner',
 Param(parent='NerDLModel_d4424c9af5f4', name='storageRef', doc='unique reference name for identification'): 'glove_100d'}

In [ ]:
empty_df= spark.createDataFrame([[" "]]).toDF("text")
pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           tokenizer,
                           word_embedding,
                           public_ner
])

pipelineModel= pipeline.fit(empty_df)
result= pipelineModel.transform(news_df.limit(100))

In [ ]:
result.show(5, truncate=30)

+--------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+
|category|                          text|                      document|                         token|                     embedding|                           ner|
+--------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+
|Business|Unions representing workers...|[[document, 0, 126, Unions ...|[[token, 0, 5, Unions, [sen...|[[word_embeddings, 0, 5, Un...|[[named_entity, 0, 5, O, [w...|
|Sci/Tech| TORONTO, Canada    A secon...|[[document, 0, 222,  TORONT...|[[token, 1, 7, TORONTO, [se...|[[word_embeddings, 1, 7, TO...|[[named_entity, 1, 7, B-LOC...|
|Sci/Tech| A company founded by a che...|[[document, 0, 209,  A comp...|[[token, 1, 1, A, [sentence...|[[word_embeddings, 1, 1, A,...|[[named_entity, 1, 1, O, [w...|
|Sci

In [ ]:
result_df= result.select(F.explode(F.arrays_zip("token.result", "ner.result")).alias("col"))\
                 .select(F.expr("col['0']").alias("token"),
                         F.expr("col['1']").alias("ner")).toPandas()
result_df.head(50)

### Onto Ner

In [ ]:
onto_ner= NerDLModel.pretrained("onto_100", "en")\
    .setInputCols(["document", "token", "embedding"])\
    .setOutputCol("onto_ner")

pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           tokenizer,
                           word_embedding,
                           onto_ner
])

model= pipeline.fit(empty_df)
result= model.transform(news_df.limit(100))

onto_100 download started this may take some time.
Approximate size to download 13.5 MB
[OK!]


In [ ]:
result_df= result.select(F.explode(F.arrays_zip("token.result" ,"onto_ner.result")).alias("col"))\
    .select(F.expr("col['0']").alias("token"),
            F.expr("col['1']").alias("onto_ner")).toPandas()
result_df.head(50)

In [ ]:
lp_model= LightPipeline(model)
lp_result= lp_model.annotate("Ahmet Emin is a good person. He lives in Bursa and I saw him at the Walmart yesterday, when we come across today it was too late to say him that buy me a orange juice.")

In [ ]:
list(zip(lp_result["token"], lp_result["onto_ner"]))

[('Ahmet', 'B-PERSON'),
 ('Emin', 'I-PERSON'),
 ('is', 'O'),
 ('a', 'O'),
 ('good', 'O'),
 ('person', 'O'),
 ('.', 'O'),
 ('He', 'O'),
 ('lives', 'O'),
 ('in', 'O'),
 ('Bursa', 'B-GPE'),
 ('and', 'O'),
 ('I', 'O'),
 ('saw', 'O'),
 ('him', 'O'),
 ('at', 'O'),
 ('the', 'O'),
 ('Walmart', 'B-ORG'),
 ('yesterday', 'B-DATE'),
 (',', 'O'),
 ('when', 'O'),
 ('we', 'O'),
 ('come', 'O'),
 ('across', 'O'),
 ('today', 'B-DATE'),
 ('it', 'O'),
 ('was', 'O'),
 ('too', 'O'),
 ('late', 'O'),
 ('to', 'O'),
 ('say', 'O'),
 ('him', 'O'),
 ('that', 'O'),
 ('buy', 'O'),
 ('me', 'O'),
 ('a', 'O'),
 ('orange', 'O'),
 ('juice', 'O'),
 ('.', 'O')]

In [ ]:
# ner with Bert
bert_embedding= BertEmbeddings.pretrained("bert_base_cased")\
    .setInputCols(["document", "token"])\
    .setOutputCol("bert")

ner_bert= NerDLModel.pretrained("ner_dl_bert", "en")\
    .setInputCols(["document", "token", "bert"])\
    .setOutputCol("ner_bert")

pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           tokenizer,
                           bert_embedding,
                           ner_bert
])

model= pipeline.fit(empty_df)
light_model= LightPipeline(model)
light_result= light_model.annotate("Ahmet Emin is a good person. He lives in Bursa and I saw him at the Walmart yesterday, when we come across today it was too late to say him that buy me a orange juice.")

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]
ner_dl_bert download started this may take some time.
Approximate size to download 15.4 MB
[OK!]


In [ ]:
list(zip(light_result["token"], light_result["ner_bert"]))

## Getting the NER Chunks with NER Converter

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer= Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

embedding= WordEmbeddingsModel.pretrained("glove_100d")\
    .setInputCols(["document", "token"])\
    .setOutputCol("embedding")

ner= NerDLModel.pretrained("onto_100", "en")\
    .setInputCols(["document", "token", "embedding"])\
    .setOutputCol("ner")

ner_chunk= NerConverter()\
    .setInputCols(["document", "token", "ner"])\
    .setOutputCol("ner_chunk")

pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           tokenizer,
                           embedding,
                           ner,
                           ner_chunk
])

empty_df= spark.createDataFrame([[" "]]).toDF("text")
model= pipeline.fit(empty_df)
result= model.transform(news_df.limit(100))

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
onto_100 download started this may take some time.
Approximate size to download 13.5 MB
[OK!]


In [ ]:
result.show(5, truncate=20)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|category|                text|            document|               token|           embedding|                 ner|           ner_chunk|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Business|Unions representi...|[[document, 0, 12...|[[token, 0, 5, Un...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 31, 45, ...|
|Sci/Tech| TORONTO, Canada ...|[[document, 0, 22...|[[token, 1, 7, TO...|[[word_embeddings...|[[named_entity, 1...|[[chunk, 1, 7, TO...|
|Sci/Tech| A company founde...|[[document, 0, 20...|[[token, 1, 1, A,...|[[word_embeddings...|[[named_entity, 1...|[[chunk, 48, 75, ...|
|Sci/Tech| It's barely dawn...|[[document, 0, 26...|[[token, 1, 4, It...|[[word_embeddings...|[[named_entity, 1...|[[chunk, 23, 38, ...|
|Sci/Tech| Southern Califor...|[[document

In [ ]:
result.select(F.explode(F.arrays_zip("ner_chunk.result", "ner_chunk.metadata")).alias("col"))\
    .select(F.expr("col['0']").alias("ner_chunk"),
            F.expr("col['1']['entity']").alias("ner_label")).show(20, truncate=False)

+-----------------------------------+-----------+
|ner_chunk                          |ner_label  |
+-----------------------------------+-----------+
|Turner   Newall                    |ORG        |
|Federal Mogul                      |ORG        |
|TORONTO                            |GPE        |
|Canada                             |GPE        |
|second                             |ORDINAL    |
|#36;10 million                     |CARDINAL   |
|Ansari X Prize                     |WORK_OF_ART|
|first                              |ORDINAL    |
|the University of Louisville       |ORG        |
|Mike Fitzpatrick                   |PERSON     |
|the day                            |DATE       |
|Southern California's              |LOC        |
|Friday                             |DATE       |
|first                              |ORDINAL    |
|British                            |NORP       |
|Department for Education and Skills|ORG        |
|""Music Manifesto""                |WORK_OF_ART|


In [ ]:
!cd ~/cache_pretrained && ls -1

glove_100d_en_2.4.0_2.4_1579690104032
onto_100_en_2.4.0_2.4_1579729071672


In [ ]:
!cd ~/cache_pretrained && pwd

/root/cache_pretrained


## Sparknlp Visualizer

In [ ]:
%%capture
! pip install spark-nlp-display

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

In [ ]:
pipeline= PretrainedPipeline("recognize_entities_dl", lang="en")

recognize_entities_dl download started this may take some time.
Approx size to download 160.1 MB
[OK!]


In [ ]:
model= pipeline.fullAnnotate("Ahmet Emin is a good person. He lives in Bursa and I saw him at the Walmart yesterday, when we come across today it was too late to say him that buy me a orange juice.")[0]
model.keys()

dict_keys(['entities', 'document', 'token', 'ner', 'embeddings', 'sentence'])

In [ ]:
from sparknlp_display import NerVisualizer

In [ ]:
visualiser = NerVisualizer()
visualiser.display(model, label_col='entities', document_col='document')

# Change color of an entity label
visualiser.set_label_colors({'LOC':'#008080', 'PER':'#800080'})
visualiser.display(model, label_col='entities')

# Set label filter
visualiser.display(model, label_col='entities', document_col='document',
                   labels=['PER'])

print ('\nColor code for label: \n"LOC": {}\n"PER": {}' .format(visualiser.get_label_color('LOC'),visualiser.get_label_color('PER')) )


Color code for label: 
"LOC": #008080
"PER": #800080


In [ ]:
visualiser = NerVisualizer()
visualiser.display(model, label_col='entities', document_col='document')

## Using Pretrained ClassifierDL and SentimentDL models¶


In [ ]:
fake_classifier= ClassifierDLModel.pretrained("classifierdl_use_fakenews", "en")\
    .setInputCols(["document", "sentence_embedding"])\
    .setOutputCol("clasifier")
  

classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[OK!]


In [ ]:
fake_classifier.getClasses()

['FAKE', 'REAL']

In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use= UniversalSentenceEncoder.pretrained(lang="en")\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embedding")

pipeline= Pipeline(stages=[ 
                           documentAssembler,
                           use,
                           fake_classifier
])

model= pipeline.fit(empty_df)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [ ]:
text = 'BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump'
spark_df= spark.createDataFrame([[text]]).toDF("text")
result= model.transform(spark_df)

In [ ]:
result.show(5, truncate=30)

+------------------------------+------------------------------+------------------------------+------------------------------+
|                          text|                      document|            sentence_embedding|                     clasifier|
+------------------------------+------------------------------+------------------------------+------------------------------+
|BREAKING: Leaked Picture Of...|[[document, 0, 96, BREAKING...|[[sentence_embeddings, 0, 9...|[[category, 0, 96, FAKE, [s...|
+------------------------------+------------------------------+------------------------------+------------------------------+



In [ ]:
result.select("text", "clasifier.result").show(truncate=False)

+-------------------------------------------------------------------------------------------------+------+
|text                                                                                             |result|
+-------------------------------------------------------------------------------------------------+------+
|BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump|[FAKE]|
+-------------------------------------------------------------------------------------------------+------+



In [ ]:
def get_clf_lp(model_name, sentiment_dl=False, pretrained=True):

  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  use = UniversalSentenceEncoder.pretrained(lang="en") \
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")


  if pretrained:

    if sentiment_dl:

      document_classifier = SentimentDLModel.pretrained(model_name, 'en') \
                .setInputCols(["document", "sentence_embeddings"]) \
                .setOutputCol("class")
    else:
      document_classifier = ClassifierDLModel.pretrained(model_name, 'en') \
                .setInputCols(["document", "sentence_embeddings"]) \
                .setOutputCol("class")

  else:

    if sentiment_dl:

      document_classifier = SentimentDLModel.load(model_name) \
                .setInputCols(["document", "sentence_embeddings"]) \
                .setOutputCol("class")
    else:
      document_classifier = ClassifierDLModel.load(model_name) \
                .setInputCols(["document", "sentence_embeddings"]) \
                .setOutputCol("class")

  print ('classes:',document_classifier.getClasses())

  nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      use,
      document_classifier
  ])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  clf_pipelineFit = nlpPipeline.fit(empty_data)

  clf_lp_pipeline = LightPipeline(clf_pipelineFit)

  return clf_lp_pipeline


In [ ]:
clf_lp_pipeline= get_clf_lp("classifierdl_use_fakenews")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[OK!]
classes: ['FAKE', 'REAL']


In [ ]:
clf_lp_pipeline.annotate("Italy has won the 2021 European Cup")

{'class': ['FAKE'],
 'document': ['Italy has won the 2021 European Cup'],
 'sentence_embeddings': ['Italy has won the 2021 European Cup']}

In [ ]:
!cd ~/cache_pretrained && ls -1

classifierdl_use_emotion_en_2.7.1_2.4_1610190563302
classifierdl_use_fakenews_en_2.7.1_2.4_1610187399147
classifierdl_use_trec50_en_2.7.1_2.4_1610118328412
glove_100d_en_2.4.0_2.4_1579690104032
onto_100_en_2.4.0_2.4_1579729071672
recognize_entities_dl_en_3.0.0_3.0_1616473647254
tfhub_use_en_2.4.0_2.4_1587136330099
